# Imports

In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels
import statsmodels.sandbox.stats.multicomp
import pandas as pd
import datetime
from statsmodels.stats.weightstats import ztest

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm
from sklearn.utils import shuffle
import hashlib
from base64 import b64encode
import collections

import warnings
warnings.filterwarnings('ignore')

# Download data

In [2]:
# import requests

# # download dataset by chunks
# url = "https://github.com/sharthZ23/your-second-recsys/raw/master/data_kion.zip"

# req = requests.get(url, stream=True)

# with open('data_kion.zip', "wb") as fd:
#     total_size_in_bytes = int(req.headers.get('Content-Length', 0))
#     progress_bar = tqdm(desc='kion dataset download', total=total_size_in_bytes, unit='iB', unit_scale=True)
#     for chunk in req.iter_content(chunk_size=2 ** 20):
#         progress_bar.update(len(chunk))
#         fd.write(chunk)

In [3]:
# # Распаковываем архив из 3 файлов
# import zipfile
 
# with zipfile.ZipFile('data_kion.zip', 'r') as zip_ref:
#     zip_ref.extractall()

In [4]:
interactions = pd.read_csv('../data/interactions_df.csv')
interactions.head()

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0


# Задание

## ДЗ 1:

### Дизайн теста
1) Из interactions взять данные за 2 последние недели (2021-08-09 по 2021-08-22)

2) Посчитайте на этих данных корректность на бутстрепе и сравните ее с результатами за 1 неделю (посчитано на семинаре) (0.5 балла)

3) На двух неделях посчитайте мощность. Для этого нужно в одну из групп докинуть эффект. Докиньте эффект в 1%, 3% и 5% и сравните полученную мощность (1.5 балла)

4) Посчитайте MDE, который можно зафиксировать на 2х неделях. В качестве alpha и beta подставьте ваши вычисленные ошибки 1 и 2 рода. Учитывайте что у нас формула для MDE работает для t-test или z-test (1 балл)

### A/B тест
1) Представим что у нас прошел тест, используем те же самые данные за 2 недели

2) Занулите для всех пользователей total_dur, у которых total_dur < 500. Их НЕ УБИРАЕМ, а просто обрабатываем эти значения, принимая за нулевые, но пользователей также учитываем в эксперименте (0.5 балла)

3) Разбейте их самостоятельно на две равные группы, используйте функцию groups_splitter и соль = 'kiontestmodel20210805' (0.5 балла)

4) Оказалось, что модель в группе В показала себя лучше, чем в группе А на 2.5%, причем эффект распространился неравномерно и преимущественно на 10% самых смотрящих пользователей, докиньте такой эффект самостоятельно. Нужно метрику total_dur увеличить на 2.5% для 10% пользователей с самым продолжительным смотрением. (1 балл)

5) Посчитайте результат такого теста и сделайте выводы (2 балла)

Еще 1 балл можно получить за хорошее и подробное объяснение каждого шага 

# Решение

In [5]:
# предобработка данных
interactions['last_watch_dt'] = pd.to_datetime(interactions['last_watch_dt']).map(lambda x: x.date())
interactions['user_id'] = interactions['user_id'].astype(str)

print(f"Уникальных юзеров в interactions: {interactions['user_id'].nunique():_}")
print(f"Уникальных айтемов в interactions: {interactions['item_id'].nunique():_}")

Уникальных юзеров в interactions: 962_179
Уникальных айтемов в interactions: 15_706


In [6]:
# выделим данные за 2 последние недели
weeks = interactions[interactions['last_watch_dt'] >= interactions['last_watch_dt'].max() - datetime.timedelta(days=2*7)]
weeks = weeks.groupby('user_id', as_index=False).agg({'total_dur': sum})

In [7]:
# Делим на 2 равные группы 
def groups_splitter(df, columns, user_salt=None):
    
    if user_salt == None:
        salt = salt_generator()
    else:
        salt = user_salt
    
    df['hash'] = ((df['user_id'].astype(str)) + '#' + salt).apply(lambda x: hashlib.sha256(x.encode('utf-8')).hexdigest())

    df['group'] = ((df['hash'].str.slice(start=-6).apply(int, base=16) % 2).map(lambda x: 'A' if x == 0 else 'B'))

    return df[columns].drop_duplicates()

In [8]:
def salt_generator(salt=None):
    import os
    from base64 import b64encode
    salt = os.urandom(8)
    
    return b64encode(salt).decode('ascii')


def get_bootstrap_array(arr):
    return np.random.choice(arr, replace=True, size=len(arr))


def calc_bootstrap_mean(arr, size=1000):
    result = np.empty(size)
    for i in range(size):
        result[i] = np.mean(get_bootstrap_array(arr))
    return result

In [9]:
def get_p_value(dist):
    dist = np.array(dist)
    x = (dist > 0).mean()
    pvalue = min(x, 1 - x) * 2
    return pvalue

```text
significance: 5.0% - на семинаре
```

In [ ]:
correctness = []
values = []

for i in tqdm(range(1000)): # в дз используем 1000 итераций 
    
    new_df = groups_splitter(weeks.copy(), columns=['user_id', 'total_dur', 'group'], user_salt=salt_generator()).drop_duplicates()

    
    vec_a = new_df[(new_df['group'] == 'A')]['total_dur']
    vec_b = new_df[(new_df['group'] == 'B')]['total_dur']
    
    #bootstrap
    sample_a_mean = calc_bootstrap_mean(vec_a, size=1000) # на бутстрэпе считаем средние для вектора
    sample_b_mean = calc_bootstrap_mean(vec_b, size=1000)
    values.append(sample_a_mean - sample_b_mean)
    
    left_side, right_side = np.percentile(sample_a_mean - sample_b_mean, 
                                                  [100 * 0.05 / 2., 100 * (1 - 0.05 / 2.)]) # считаем 95% доверительный интервал для разницы средних
    
    correctness.append(not left_side <= 0 <= right_side)
    
    test_correctness = collections.Counter(correctness)
    
print(f'alpha = {test_correctness[1]/(test_correctness[1] + test_correctness[0])*100}%')

Полученные результаты

0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [1:16:56<00:00,  4.62s/it]

$\alpha$ = 4.3%

### Мощность

In [17]:
powerness = []
values = []
power = [1.01, 1.03, 1.05]
beta = []
for _power in power:
    for i in tqdm(range(100)): # в дз используем 1000 итераций 
        new_df = groups_splitter(weeks.copy(), columns=['user_id', 'total_dur', 'group'], user_salt=salt_generator()).drop_duplicates()

        
        vec_a = new_df[(new_df['group'] == 'A')]['total_dur']
        vec_b = new_df[(new_df['group'] == 'B')]['total_dur'] * _power
        
        #bootstrap
        sample_a_mean = calc_bootstrap_mean(vec_a, size=1000) # на бутстрэпе считаем средние для вектора
        sample_b_mean = calc_bootstrap_mean(vec_b, size=1000)
        values.append(sample_a_mean - sample_b_mean)
        
        left_side, right_side = np.percentile(sample_a_mean - sample_b_mean, 
                                                    [100 * 0.05 / 2., 100 * (1 - 0.05 / 2.)]) # считаем 95% доверительный интервал для разницы средних
        
        powerness.append(left_side <= 0 <= right_side)
        
    test_power = collections.Counter(powerness)
    beta.append(test_power[1]/(test_power[1] + test_power[0])*100)
    
for _beta in beta:
    print(f'beta = {_beta}%')

100%|██████████| 100/100 [07:54<00:00,  4.75s/it]

beta = 89.0%
beta = 68.0%
beta = 46.0%


При добавлении N% эффекта, где N:
- 1% - $\beta$ = 89.0%
- 3% - $\beta$ = 68.0%
- 5% - $\beta$ = 46.0%

In [10]:
def get_mde(metric_vec, alpha=0.05, beta=0.2):
    
    metric_mean, metric_std, metric_n = metric_vec.mean(), metric_vec.std(), metric_vec.count()
    
    z_alpha = stats.norm.ppf(1 - (alpha / 2), loc=0, scale=1)
    z_beta = stats.norm.ppf(1 - beta, loc=0, scale=1)
    
    mde = (z_alpha + z_beta)*metric_std / np.sqrt(metric_n)
    
    return mde*100/metric_mean

In [ ]:
get_mde(new_df.total_dur, alpha=0.043, beta=0.46)

MDE = 1.37% - детектируемое изменение


2) Занулите для всех пользователей total_dur, у которых total_dur < 500. Их НЕ УБИРАЕМ, а просто обрабатываем эти значения, принимая за нулевые, но пользователей также учитываем в эксперименте (0.5 балла)


In [ ]:
interactions.loc[interactions["total_dur"] < 500,"total_dur"] = 0

In [ ]:
interactions.sort_values(["total_dur"]).head()

,user_id,item_id,last_watch_dt,total_dur,watched_pct
3410681,797608,10886,2021-07-13,0,3.0
1932470,330119,7025,2021-07-27,0,1.0
2632292,511161,6003,2021-08-11,0,4.0
3178082,792786,7626,2021-07-20,0,1.0
2801782,235733,12401,2021-06-08,0,2.0


3) Разбейте их самостоятельно на две равные группы, используйте функцию groups_splitter и соль = 'kiontestmodel20210805' (0.5 балла)

4) Оказалось, что модель в группе В показала себя лучше, чем в группе А на 2.5%, причем эффект распространился неравномерно и преимущественно на 10% самых смотрящих пользователей, докиньте такой эффект самостоятельно. Нужно метрику total_dur увеличить на 2.5% для 10% пользователей с самым продолжительным смотрением. (1 балл)


In [ ]:
splitted_df = groups_splitter(weeks.copy(),columns=['user_id', 'total_dur', 'group'],user_salt="kiontestmodel20210805").drop_duplicates()
splitted_df.loc[
    splitted_df.sort_values(["total_dur"], ascending=False).head(int(splitted_df.shape[0] * 0.1)).index,
    ["total_dur"]
] *= 1.025
splitted_df["total_dur"] = splitted_df["total_dur"].astype(int)
# splitted_df.sort_values(["total_dur"], ascending=False)

5) Посчитайте результат такого теста и сделайте выводы (2 балла)

In [23]:
powerness = []
values = []
for i in tqdm(range(1000)): # в дз используем 1000 итераций 
    splitted_df = groups_splitter(weeks.copy(),columns=['user_id', 'total_dur', 'group'],user_salt="kiontestmodel20210805").drop_duplicates()

    vec_a = splitted_df[(splitted_df['group'] == 'A')]['total_dur']
    vec_b = splitted_df[(splitted_df['group'] == 'B')]['total_dur'].to_frame()
    vec_b.loc[
        vec_b.sort_values(["total_dur"], ascending=False).head(int(vec_b.shape[0] * 0.1)).index,
        ["total_dur"]
    ] *= 1.025
    vec_b["total_dur"] = vec_b["total_dur"].astype(int)
    vec_b = vec_b['total_dur']
    
    #bootstrap
    sample_a_mean = calc_bootstrap_mean(vec_a, size=1000) # на бутстрэпе считаем средние для вектора
    sample_b_mean = calc_bootstrap_mean(vec_b, size=1000)
    values.append(sample_a_mean - sample_b_mean)
    
    left_side, right_side = np.percentile(sample_a_mean - sample_b_mean, 
                                                [100 * 0.08 / 2., 100 * (1 - 0.08 / 2.)]) # считаем 95% доверительный интервал для разницы средних
    
    powerness.append(left_side <= 0 <= right_side)
    
    test_powerness = collections.Counter(powerness)
beta = (test_powerness[1]/(test_powerness[1] + test_powerness[0])*100)
print(f'beta = {beta}%')

100%|██████████| 1000/1000 [1:19:15<00:00,  4.76s/it]

beta = 100.0%


In [12]:
from scipy.stats import ttest_ind
from statsmodels.stats.weightstats import ztest

In [13]:
splitted_df = groups_splitter(weeks.copy(),columns=['user_id', 'total_dur', 'group'],user_salt="kiontestmodel20210805").drop_duplicates()

vec_a = splitted_df[(splitted_df['group'] == 'A')]['total_dur']
vec_b = splitted_df[(splitted_df['group'] == 'B')]['total_dur'].to_frame()
vec_b.loc[
    vec_b.sort_values(["total_dur"], ascending=False).head(int(vec_b.shape[0] * 0.1)).index,
    ["total_dur"]
] *= 1.025
vec_b["total_dur"] = vec_b["total_dur"].astype(int)
vec_b = vec_b['total_dur']

ttest_ind(vec_a, vec_b).pvalue

0.28852897564113666

In [14]:
metric, pval = ztest(vec_b, vec_a, alternative='larger')
pval

0.14426402866664528